In [5]:

# GUI Libraries
import tkinter
import tkinter.messagebox
import customtkinter

# App libraries
from PIL import Image, ImageEnhance
import cv2
from dsia_lib import dsia
import pandas as pd
import numpy as np
from tkinter import filedialog as fd


# estados de la app
null = -1
started = 0
detected = 1
classified = 2
routed = 3
executed = 4

class App(customtkinter.CTk):
    def __init__(self):
        
        super().__init__()

        # reads configuration file
        self.cfg = pd.read_json('dsia_conf.json')
        
        # create drosophila IA class
        self.ds = dsia(self.cfg.classify.model)
        
        # configure app
        self.title("DSIA")
        self.state('zoomed')
        #self.attributes('-fullscreen', True)
        #self.iconbitmap('.ico')

        # configure grid main layout (3x1)
        self.grid_columnconfigure(0, weight = 0)    # column 1
        self.grid_columnconfigure(1, weight = 1)    # column 2
        self.grid_columnconfigure(2, weight = 0)    # column 3
        self.grid_rowconfigure(0, weight=1)         # row 0

        # set left panel
        self.set_left_panel()

        # set middle panel
        self.set_middle_panel()
                
        # set right panel
        self.set_right_panel()

        # configuration parameters
        self.set_params()

        # set button status
        self.set_screen()

    def set_params(self):

        # set theme and color options
        customtkinter.set_appearance_mode(self.cfg.appearance_mode[0])          # Modes: "System" (standard), "Dark", "Light"
        customtkinter.set_default_color_theme(self.cfg.color_theme[0])          # Themes: "blue" (standard), "green", "dark-blue"

        self.status = null

        if self.cfg.checkbox.bbox==1: self.cb_bbox.toggle()
        if self.cfg.checkbox.objn==1: self.cb_objn.toggle()
        if self.cfg.checkbox.conf==1: self.cb_conf.toggle()
        if self.cfg.checkbox.targ==1: self.cb_targ.toggle()
        if self.cfg.checkbox.rout==1: self.cb_rout.toggle()

    def set_left_panel(self):

        # create and locate the left panel frame
        self.left_panel = customtkinter.CTkFrame(self, corner_radius=0)
        self.left_panel.grid(row=0, column=0, sticky="news")   
        self.left_panel.grid_rowconfigure((0, 1, 2, 3, 4, 5, 6, 7, 8, 9), weight=0)                     
        self.left_panel.grid_rowconfigure(10, weight=1)
        
        # get logo image
        image = "./test_images/CustomTkinter_logo_single.png"
        
        # left panel widgets
        self.logo_image = customtkinter.CTkImage(Image.open(image), size=(26, 26))
        self.logo_label = customtkinter.CTkLabel(self.left_panel, text="  DSIA", image=self.logo_image, compound="left", font=customtkinter.CTkFont(size=20, weight="bold"))
        self.load_btn = customtkinter.CTkButton(self.left_panel, text="Cargar imagen", command=self.image_load)
        self.take_btn = customtkinter.CTkButton(self.left_panel, text="Tomar foto", command=self.take_image)
        self.detect_btn = customtkinter.CTkButton(self.left_panel, text = "Detectar", command=self.detect)
        self.classify_btn = customtkinter.CTkButton(self.left_panel, text = "Clasificar", command=self.classify)
        self.route_btn = customtkinter.CTkButton(self.left_panel, text = "Trayectoria", command=self.route)
        self.process_btn = customtkinter.CTkButton(self.left_panel, text = "Procesar", command=self.laser_process)
        self.clear_btn = customtkinter.CTkButton(self.left_panel, text = "Borrar", command=self.clear, hover_color="#EA999E", fg_color="#C46872")
        self.config_btn = customtkinter.CTkButton(self.left_panel, text = "Configuración", command=self.config)

        # left panel widget position
        self.logo_label.grid(row=0, column=0, padx=20, pady=(20, 10), sticky = "new")
        self.load_btn.grid(row=1, column=0, padx=20, pady=10, sticky = "new")
        self.take_btn.grid(row=2, column=0, padx=20, pady=10, sticky = "new")
        self.detect_btn.grid(row=3, column=0, padx=20, pady=10, sticky = "new")
        self.classify_btn.grid(row=4, column=0, padx=20, pady=10, sticky = "new") 
        self.route_btn.grid(row=5, column=0, padx=20, pady=10, sticky = "new")
        self.process_btn.grid(row=6, column=0, padx=20, pady=10, sticky = "new")
        self.clear_btn.grid(row=7, column=0, padx=20, pady=10, sticky = "new")
        
        
        self.config_btn.grid(row=10, column=0, padx=20, pady=(20,20), sticky = "sew")

    def set_middle_panel(self):

        # create and locate the midle panel 
        self.mid_panel = customtkinter.CTkFrame(self, corner_radius=0)
        self.mid_panel.grid(row=0, column=1, sticky="nsew")   
        self.mid_panel.grid_columnconfigure(0, weight = 1)
        self.mid_panel.grid_rowconfigure(0, weight = 0) 
        self.mid_panel.grid_rowconfigure(1, weight = 1)
        self.mid_panel.grid_rowconfigure(2, weight = 0)
        self.mid_panel.grid_rowconfigure(3, weight = 0) 

        # image title
        self.image_title = customtkinter.CTkLabel(self.mid_panel, text="[sin imagen]", text_color="grey")
        self.image_title.grid(row=0, column=0, padx=(10, 10), pady=(30, 5), sticky="nw")

        # main image area
        self.main_image = customtkinter.CTkLabel(self.mid_panel, text="", bg_color="#5E5E5E")
        self.main_image.grid(row=1, column=0, padx=(10, 10), pady=(0, 10), sticky="nsew")
        # Asocia la función de click
        self.main_image.bind("<Button-1>", self.left_click_img)
        self.main_image.bind("<Double-Button-1>", self.doble_left_click_img)

        # information panel
        self.information_panel = customtkinter.CTkFrame(self.mid_panel, fg_color="black")
        self.information_panel.grid(row=2, column=0, padx=(10, 10), pady=(10, 10), sticky="nsew")
        self.information_panel.grid_columnconfigure(0, weight=1)
        self.information_panel.grid_columnconfigure(1, weight=1)
        self.information_panel.grid_columnconfigure(2, weight=1)
        self.information_panel.grid_rowconfigure((0, 1, 2, 3, 4), weight=1)

        # information widgets
        self.label_status = customtkinter.CTkLabel(master=self.information_panel, text="Estado:", anchor="nw")
        self.label_objects_count = customtkinter.CTkLabel(master=self.information_panel, text="# detectados: 0", anchor="nw")
        self.label_objects_to_classify = customtkinter.CTkLabel(master=self.information_panel, text="# por clasificar: 0", anchor="nw")
        self.label_objects_overlaped = customtkinter.CTkLabel(master=self.information_panel, text="# superpuestos: 0", anchor="nw")
        self.label_objects_outside1 = customtkinter.CTkLabel(master=self.information_panel, text="# fuera umbral: 0", anchor="nw")
        self.label_objects_classified = customtkinter.CTkLabel(master=self.information_panel, text="# clasificados: 0", anchor="nw")
        self.label_objects_not_classified = customtkinter.CTkLabel(master=self.information_panel, text="no clasificados: 0", anchor="nw")        
        self.label_machos = customtkinter.CTkLabel(master=self.information_panel, text="machos: 0", anchor="nw")
        self.label_hembras = customtkinter.CTkLabel(master=self.information_panel, text="hembras: 0", anchor="nw")
        self.label_objects_to_process = customtkinter.CTkLabel(master=self.information_panel, text="individuos a procesar: 0", anchor="nw")


        self.cb_bbox = customtkinter.CTkCheckBox(master=self.information_panel, text="Bounding Boxes", border_width=1, checkbox_height=20, checkbox_width=20, corner_radius=4, hover=False, bg_color="black", fg_color="black", border_color="black", command=self.checkbox_update)
        self.cb_objn = customtkinter.CTkCheckBox(master=self.information_panel, text="Object numbert", border_width=1, checkbox_height=20, checkbox_width=20, corner_radius=4, hover=False, bg_color="black", fg_color="black", border_color="black", command=self.checkbox_update)
        self.cb_conf = customtkinter.CTkCheckBox(master=self.information_panel, text="Confidence", border_width=1, checkbox_height=20, checkbox_width=20, corner_radius=4, hover=False, bg_color="black", fg_color="black", border_color="black", command=self.checkbox_update)
        self.cb_targ = customtkinter.CTkCheckBox(master=self.information_panel, text="Target", border_width=1, checkbox_height=20, checkbox_width=20, corner_radius=4, hover=False, bg_color="black", fg_color="black", border_color="black", command=self.checkbox_update)
        self.cb_rout = customtkinter.CTkCheckBox(master=self.information_panel, text="Trazado", border_width=1, checkbox_height=20, checkbox_width=20, corner_radius=4, hover=False, bg_color="black", fg_color="black", border_color="black", command=self.checkbox_update)


        self.label_status.grid(row=0, column = 0, padx=(20, 10), pady=(5, 5), sticky="ew")
        self.label_objects_count.grid(row=1, column = 0, padx=(20, 10), pady=(5, 5), sticky="ew")
        self.label_objects_to_classify.grid(row=2, column = 0, padx=(20, 10), pady=(5, 5), sticky="ew")
        self.label_objects_overlaped.grid(row=3, column = 0, padx=(20, 10), pady=(5, 5), sticky="ew")
        self.label_objects_outside1.grid(row=4, column = 0, padx=(20, 10), pady=(5, 5), sticky="ew")
        #self.label_objects_outside2.grid(row=4, column = 0, padx=(20, 10), pady=(5, 5), sticky="ew")

        self.label_objects_classified.grid(row=0, column = 1, padx=(20, 10), pady=(5, 5), sticky="ew")
        self.label_objects_not_classified.grid(row=1, column = 1, padx=(20, 10), pady=(5, 5), sticky="ew")
        self.label_machos.grid(row=2, column = 1, padx=(20, 10), pady=(5, 5), sticky="ew")
        self.label_hembras.grid(row=3, column = 1, padx=(20, 10), pady=(5, 5), sticky="ew")
        self.label_objects_to_process.grid(row=4, column = 1, padx=(20, 10), pady=(5, 5), sticky="ew")

        self.cb_bbox.grid(row=0, column = 2, padx=(20, 10), pady=(5, 5), sticky="ew")
        self.cb_objn.grid(row=1, column = 2, padx=(20, 10), pady=(5, 5), sticky="ew")
        self.cb_conf.grid(row=2, column = 2, padx=(20, 10), pady=(5, 5), sticky="ew")
        self.cb_targ.grid(row=3, column = 2, padx=(20, 10), pady=(5, 5), sticky="ew")
        self.cb_rout.grid(row=4, column = 2, padx=(20, 10), pady=(5, 5), sticky="ew")


        # LINEA DE CHAT ABAJO
        
        self.entry = customtkinter.CTkEntry(self.mid_panel, placeholder_text="CTkEntry")
        self.entry.grid(row=3, column=0, padx=(20, 0), pady=(20, 20), sticky="nsew")

    def checkbox_update(self):
        # configure check boxes
        self.ds.draw_bbox = (1 if (self.cb_bbox.get()==1) else 0)
        self.ds.draw_objn = (1 if (self.cb_objn.get()==1) else 0)
        self.ds.draw_conf = (1 if (self.cb_conf.get()==1) else 0)
        self.ds.draw_targ = (1 if (self.cb_targ.get()==1) else 0)
        self.ds.draw_rout = (1 if (self.cb_rout.get()==1) else 0)

        # redibuja la pantalla
        if self.status in (1, 2, 3, 4):
            self.set_image(self.ds.draw_insect_info())

    def set_right_panel(self):

        # create and locate the right top panel 
        self.right_panel = customtkinter.CTkFrame(self, corner_radius=0)
        self.right_panel.grid(row=0, column=2, sticky="news")   
        self.right_panel.grid_columnconfigure(0, weight = 1)
        self.right_panel.grid_rowconfigure(0, weight = 0) 
        self.right_panel.grid_rowconfigure(1, weight = 0) 
        self.right_panel.grid_rowconfigure(2, weight = 0) 
        self.right_panel.grid_rowconfigure(3, weight = 1) 

        # insect number id
        self.insect_id = customtkinter.CTkLabel(self.right_panel, text="[no id]", text_color="grey")
        self.insect_id.grid(row=0, column=0, padx=(0, 10), pady=(30, 5), sticky="nw")

        # zoom insect image
        self.insect_view = customtkinter.CTkLabel(self.right_panel, width=300, height=300, text="", bg_color="#5E5E5E")
        self.insect_view.grid(row=1, column=0, pady=(0,0), padx=(0,10), sticky="new")

        self.insect_sex = customtkinter.CTkButton(master=self.insect_view, 
                                                  fg_color="transparent", 
                                                  text="n/a", 
                                                  width=100, 
                                                  height=20,
                                                  border_width=1, 
                                                  #text_color=("gray10", "#DCE4EE"), 
                                                  font=customtkinter.CTkFont(size=10, weight="bold"),
                                                  command=self.change_sex, 
                                                  corner_radius=0,
                                                  state="disabled"
                                                  )
        

        self.insect_sex.place(in_=self.insect_view, 
                              rely=0.92, 
                              relx=0.009)

    

        self.insect_sex.place_forget()
        


        # insect data panel
        self.insect_data = customtkinter.CTkFrame(self.right_panel, corner_radius=4)
        self.insect_data.grid(row=2, column=0, sticky="news")   
        self.insect_data.grid_columnconfigure(0, weight = 1)
        self.insect_data.grid_rowconfigure(0, weight = 1) 
        self.insect_data.grid_rowconfigure(1, weight = 1) 
        self.insect_data.grid_rowconfigure(2, weight = 1) 
        self.insect_data.grid_rowconfigure(3, weight = 1) 
        self.insect_data.grid_rowconfigure(4, weight = 1) 
        self.insect_data.grid_rowconfigure(5, weight = 1) 
        self.insect_data.grid_rowconfigure(6, weight = 1) 

        self.insect_status = customtkinter.CTkLabel(self.insect_data, text="estado", bg_color="red")
        #self.insect_sex = customtkinter.CTkButton(master=self.insect_data, fg_color="pink", text="n/a", width=30, border_width=2, text_color=("gray10", "#DCE4EE"), command=self.change_sex)
        self.insect_confidence = customtkinter.CTkLabel(self.insect_data, text="confianza", bg_color="red")
        self.a4 = customtkinter.CTkLabel(self.insect_data, text="superficie", bg_color="red")
        self.a5 = customtkinter.CTkLabel(self.insect_data, text="brillo", bg_color="red")
        self.bright = customtkinter.CTkSlider(self.insect_data, from_=0, to=2, number_of_steps=255, command=self.bright_change)

        self.insect_status.grid(row=0, column=0, padx=(10, 10), pady=(20, 5),sticky="ew")        
        #self.insect_sex.grid(row=1, column=0, padx=(10, 10), pady=(5, 5),sticky="ew")
        self.insect_confidence.grid(row=2, column=0, padx=(10, 10), pady=(5, 5),sticky="ew")
        self.a4.grid(row=3, column=0, padx=(10, 10), pady=(5, 5),sticky="ew")
        self.a5.grid(row=4, column=0, padx=(10, 10), pady=(5, 5),sticky="ew")
        self.bright.grid(row=5, column=0, padx=(10, 10), pady=(5, 5), sticky="ew")


        # botones
        self.button_change_sex = customtkinter.CTkButton(master=self.insect_data, fg_color="pink", text="Hembra", width=30, border_width=2, text_color=("gray10", "#DCE4EE"), command=self.change_sex)
        self.button_ommit = customtkinter.CTkButton(master=self.insect_data, fg_color="transparent", text="omitir", border_width=2, width=30, text_color=("gray10", "#DCE4EE"), command=self.insect_to_ommit)
        self.button_classify = customtkinter.CTkButton(master=self.insect_data, fg_color="transparent", text="clasificar", border_width=2, width=30, text_color=("gray10", "#DCE4EE"), command=self.insect_to_classify)

        self.button_change_sex.grid(row=6, column=0, padx=(20, 20), pady=(20, 20))
        self.button_ommit.grid(row=6, column=0, padx=(20, 20), pady=(20, 20), sticky="w")
        self.button_classify.grid(row=6, column=0, padx=(20, 20), pady=(20, 20), sticky="e")

        # boton
        self.main_button_1 = customtkinter.CTkButton(master=self.right_panel, fg_color="transparent", border_width=2, text_color=("gray10", "#DCE4EE"))
        self.main_button_1.grid(row=3, column=0, padx=(20, 20), pady=(20, 20), sticky="sew")

    def insect_to_classify(self):
        
        self.img_main = self.ds.change_insect(self.selected_id, "sin clasificar")
        
        wf, hf = self.getScaled(self.img_main)
        CtkImg = customtkinter.CTkImage(light_image = Image.fromarray(self.img_main), size=(wf,hf))
        self.main_image.configure(image = CtkImg)

    def insect_to_ommit(self):
        self.img_main = self.ds.change_insect(self.selected_id, "omitido")
        
        wf, hf = self.getScaled(self.img_main)
        CtkImg = customtkinter.CTkImage(light_image = Image.fromarray(self.img_main), size=(wf,hf))
        self.main_image.configure(image = CtkImg)

    def clear(self):
        self.status = null
        self.ds.path = self.ds.path[0:0]
        self.ds.drosophila = self.ds.drosophila[0:0]
        self.main_image.configure(image = "")
        self.set_screen()

    def change_sex(self):

        if (self.insect_sex.cget("text") == "macho"):
            self.img_main = self.ds.change_insect(self.selected_id, "f")
            self.sex_define('f')
        elif (self.insect_sex.cget("text") == "hembra"):
            self.img_main = self.ds.change_insect(self.selected_id, "m")
            self.sex_define('m')                
        
        wf, hf = self.getScaled(self.img_main)
        CtkImg = customtkinter.CTkImage(light_image = Image.fromarray(self.img_main), size=(wf,hf))
        self.main_image.configure(image = CtkImg)

    def getScaled(self, img):
        
        # Corrección de la proporción en relación con el contenedor
        rw = img.shape[1]/self.main_image._current_width
        rh = img.shape[0]/self.main_image._current_height
        e = img.shape[1]/img.shape[0]

        if rw>rh:
            wf = int(self.main_image._current_width)
            hf = int(wf/e)
        else:
            hf = int(self.main_image._current_height)
            wf = int(e*hf)
        
        # Calcula la corrección de escala
        self.scaleX = img.shape[1]/wf
        self.scaleY = img.shape[0]/hf

        return wf, hf

    def route(self):
 
        self.ds.insect_targeting(self.img_de)

        img = self.ds.route()

        wf, hf = self.getScaled(img)

        CtkImg = customtkinter.CTkImage(light_image = Image.fromarray(img), size=(wf,hf))
 
        self.main_image.configure(image = CtkImg)

        self.status = routed

        self.set_screen()

    def laser_process(self):
        pass
        
    def bright_change(self, b):
        img = Image.fromarray(self.click_img)
        img = ImageEnhance.Brightness(img)
        img = img.enhance(b)

        #img = ImageEnhance.Contrast
        #img = ImageEnhance.Sharpness
        
        self.image_insect = customtkinter.CTkImage(light_image = img, size=(self.insect_view._current_width, self.insect_view._current_height))    
        self.insect_view.configure(image = self.image_insect)
        
    def doble_left_click_img(self, event):        
        if self.status == detected:
            #sure_window = customtkinter.CTkToplevel(self)
            #sure_window.title = "Crear nueva area"
            #sure_window.resizable(False, False)
            #sure_window.geometry = "400x200"
            
            #sure_window_label1 = customtkinter.CTkLabel(sure_window, text=f"      {event.x} {event.y}      ")
            #sure_window_label1.pack()
            #sure_window.
            
            id = self.ds.drosophila.obj_id.max()+1
            pixels = -1
            targetX = int(self.scaleX*event.x)
            targetY = int(self.scaleY*event.y)
                                                
            self.ds.drosophila.loc[len(self.ds.drosophila.index)] = [id, pixels, (int(self.scaleX*event.x))-int(self.ds.rect/2), (int(self.scaleY*event.y))-int(self.ds.rect/2), "n/a", 'sin clasificar', targetX, targetY, 0.0]   
            print(self.ds.drosophila.tail())

            # redibujar boxes
            self.set_image(self.ds.draw_boxes())
            
        elif self.status == classified:
            self.change_sex()            

            # redibujar boxes
            self.set_image(self.ds.draw_boxes())

    def sex_define(self, sex):
        if sex == 'f':
            self.insect_sex.place(in_=self.insect_view, rely=0.92, relx=0.009)
            self.insect_sex.configure(text="hembra", fg_color="#FF00FF", state="normal", text_color="white", border_color="white", hover=False)
            self.button_change_sex.configure(fg_color="blue", text="Macho") 
        elif sex == 'm':
            self.insect_sex.place(in_=self.insect_view, rely=0.92, relx=0.009)
            self.insect_sex.configure(text="macho", fg_color="#0000FF", state="normal", text_color="white", border_color="white", hover=False)
            self.button_change_sex.configure(fg_color="pink", text="Hembra") 
        else:
            self.insect_sex.place_forget()
        
    def left_click_img(self, event):        

        insect = self.ds.drosophila[(self.ds.drosophila.x<int(self.scaleX*event.x)) &
                           (self.ds.drosophila.y<int(self.scaleY*event.y)) &
                           (self.ds.drosophila.y+self.ds.rect>int(self.scaleY*event.y)) &
                           (self.ds.drosophila.x+self.ds.rect>int(self.scaleX*event.x))]
        
        if insect.obj_id.count()>0:

            self.label_objects_to_process.configure(text=f"click en {insect.obj_id.item()} x: {event.x} y: {event.y} xr: {int(self.scaleX*event.x)} yr: {int(self.scaleY*event.y)}")


            self.click_img = self.ds.img_orig[insect.y.item():insect.y.item()+self.ds.rect, insect.x.item():insect.x.item()+self.ds.rect]
            self.image_insect = customtkinter.CTkImage(light_image = Image.fromarray(self.click_img), size=(self.insect_view._current_width, self.insect_view._current_height))
            self.insect_view.configure(image = self.image_insect)
            
            self.insect_status.configure(text = insect.state.item())
            self.insect_sex.configure(text = insect.sex.item())
            self.insect_confidence.configure(text = insect.conf.item())

            self.selected_id = insect.obj_id.item()
            self.insect_id.configure(text = f"[{insect.obj_id.item()}]",text_color="white")
            self.bright.set(1)
            
            self.sex_define(insect.sex.item())

    def open_input_dialog_event(self):
        dialog = customtkinter.CTkInputDialog(text="Type in a number:", title="CTkInputDialog")
        print("CTkInputDialog:", dialog.get_input())
        customtkinter.CTkInputDialog()

    def change_appearance_mode_event(self, new_appearance_mode: str):
        customtkinter.set_appearance_mode(new_appearance_mode)

    def change_scaling_event(self, new_scaling: str):
        new_scaling_float = int(new_scaling.replace("%", "")) / 100
        customtkinter.set_widget_scaling(new_scaling_float)

    def config(self):
        self.open_input_dialog_event()

    def image_load(self, event=None):

        filetypes = (
            ('fotos', '*.jpeg'),
            ('todos los archivos', '*.*')
        )

        image_file = fd.askopenfilename(
            title='Cargar una foto',
            initialdir='/Users/lucianosmith/Documents/CEIA/',
            filetypes=filetypes)

    
        self.image_title.configure(text = f"[{image_file}]",text_color="white")
        img_orig = cv2.imread(image_file, cv2.COLOR_BGR2RGB) 
        img_orig = cv2.cvtColor(img_orig, cv2.COLOR_RGB2BGR)
        self.ds.set_img(img_orig)

        self.set_image(img_orig)
        
        self.status = started
        self.set_screen()

    def set_image(self, img):
        wf, hf = self.getScaled(img)
        self.image = customtkinter.CTkImage(light_image = Image.fromarray(img), size=(wf, hf))
        self.main_image.configure(image = self.image)

    def take_image(self, event=None):

        # reemplazar por la imagen tomada de la camara
        image_file = "./Fotos 1-11-097.jpeg"
        self.image_title.configure(text = f"[{image_file}]",text_color="white")
        img_orig = cv2.imread(image_file, cv2.COLOR_BGR2RGB) 
        img_orig = cv2.cvtColor(img_orig, cv2.COLOR_RGB2BGR)
        self.ds.set_img(img_orig)
        self.set_image(img_orig)

        self.status = started        
        self.set_screen()
        
    def detect(self):

        # realiza la detección
        self.img_main, self.img_de = self.ds.insect_detection()

        # actualiza la imagen
        wf, hf = self.getScaled(self.img_main)
        CtkImg = customtkinter.CTkImage(light_image = Image.fromarray(self.img_main), size=(wf,hf))
        self.main_image.configure(image = CtkImg)
        

        
        self.label_objects_count.configure(text=f"cant. individuos: {self.ds.drosophila.obj_id.count()}")
        self.label_objects_to_classify.configure(text=f"individuos por clasificar: {self.ds.drosophila[self.ds.drosophila.state == 'sin clasificar'].obj_id.count()}")
        self.label_objects_overlaped.configure(text=f"individuos superpuestos: {self.ds.drosophila[self.ds.drosophila.state == 'overlaped'].obj_id.count()}")
        self.label_objects_outside1.configure(text=f"fuera umbral 1: {self.ds.drosophila[self.ds.drosophila.state == 'fuera umbral 1'].obj_id.count()+self.ds.drosophila[self.ds.drosophila.state == 'fuera umbral 2'].obj_id.count()}")
        self.label_objects_classified.configure(text=f"clasificados: {self.ds.drosophila[self.ds.drosophila.state == 'classified'].obj_id.count()}")

        self.status = detected

        # actualiza botones
        self.set_screen()
        
        #self.tool_button_1.configure() 
        #self.tool_button_2.configure() 
        #self.tool_button_3.configure() 

    def set_screen(self):

        if self.status == null:
            self.label_status.configure(text="Estado: [N/A]")
            # left panel buttons   HOVER = 23476D
            self.load_btn.configure(state="normal", fg_color="#3669A0", border_width = 0)
            self.take_btn.configure(state="normal", fg_color="#3669A0", border_width = 0)
            self.detect_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.classify_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.route_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.process_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.config_btn.configure(state="normal", fg_color="#3669A0", border_width = 0)

        elif self.status == started:
            self.label_status.configure(text="Estado: [Iniciado]")
            # left panel buttons
            self.load_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.take_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.detect_btn.configure(state="normal", fg_color="#3669A0", border_width = 0)
            self.classify_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.route_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.process_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.config_btn.configure(state="normal", fg_color="#3669A0", border_width = 0)

        elif self.status == detected:
            self.label_status.configure(text="Estado: [Detectado]")
            # left panel buttons
            self.load_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.take_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.detect_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.classify_btn.configure(state="normal", fg_color="#3669A0", border_width = 0)
            self.route_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.process_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.config_btn.configure(state="normal", fg_color="#3669A0", border_width = 0)

        elif self.status == classified:
            self.label_status.configure(text="Estado: [Clasificado]")
            # left panel buttons
            self.load_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.take_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.detect_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.classify_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.route_btn.configure(state="normal", fg_color="#3669A0", border_width = 0)
            self.process_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.config_btn.configure(state="normal", fg_color="#3669A0", border_width = 0)

        elif self.status == routed:
            self.label_status.configure(text="Estado: [Ruteado]")
            # left panel buttons
            self.load_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.take_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.detect_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.classify_btn.configure(state="disabled", fg_color="transparent", border_width=1)
            self.route_btn.configure(state="normal", fg_color="#3669A0", border_width = 0)
            self.process_btn.configure(state="normal", fg_color="#3669A0", border_width = 0)
            self.config_btn.configure(state="normal", fg_color="#3669A0", border_width = 0)

        elif self.status == executed:
            self.label_status.configure(text="Estado: [Procesado]")
            print("4")
        else:
            self.label_status.configure(text="Estado: [N/A]")
            print("otro")

    def classify(self):
        
        img_f = self.ds.image_classification(self.img_de)

        wf, hf = self.getScaled(img_f)
        CtkImg = customtkinter.CTkImage(light_image = Image.fromarray(img_f), size=(wf,hf))
        self.main_image.configure(image = CtkImg)


        self.label_objects_classified.configure(text=f"clasificados: {self.ds.drosophila[self.ds.drosophila.state == 'classified'].obj_id.count()}")        
        self.label_objects_not_classified.configure(text=f"no clasificados: {self.ds.drosophila[self.ds.drosophila.state == 'sin clasificar'].obj_id.count()}")
        self.label_machos.configure(text=f"machos: {self.ds.drosophila[self.ds.drosophila.sex == 'm'].obj_id.count()}")
        self.label_hembras.configure(text=f"hembras: {self.ds.drosophila[self.ds.drosophila.sex == 'f'].obj_id.count()}")
        self.label_objects_to_process.configure(text=f"individuos a procesar: {self.ds.drosophila[self.ds.drosophila.state == 'fuera umbral 2'].obj_id.count()}")
        
        self.status = classified
        # actualiza botones
        self.set_screen()

        self.img_de = img_f.copy()

if __name__ == "__main__": 
    app = App()
    app.mainloop()